In [1]:
import transformers
import json
import sys
import os
import datasets
squad_dataset = datasets.load_dataset('squad')
from datasets import concatenate_datasets, load_dataset

covid_file = '../data/COVID-QA.json'
bio_file = '../bioASQ/bioASQ.json'
config_file_location = '../data/config.json'
squad_adapt_pretrained_loc = '../models/squad_adapt_pretrained'

def make_and_save_full_dataset(train, valid, test, path):
    full_data = datasets.dataset_dict.DatasetDict({'train':train, 'validation':valid, 'test': test})
    full_data.save_to_disk(path)

def get_dataset(filename):
    return datasets.load_dataset('custom_squad.py', data_files= {'train':filename})['train']

def update_config(checkpoint, file_location = '../data/config.json'):
    config_file = open(file_location, 'r')
    config = json.load(config_file)
    print(config)
    config["_name_or_path"] = checkpoint
    config_file = open(file_location, 'w')
    json.dump(config, config_file, indent= 2)
    config_file.close()

Reusing dataset squad (C:\Users\unitu\.cache\huggingface\datasets\squad\plain_text\1.0.0\6b6c4172d0119c74515f44ea0b8262efe4897f2ddb6613e5e915840fdc309c16)


In [2]:
import run_qa

def run_gradual_ft(output_dir, checkpoint, covid_val, lr, dataset_name):
    !python run_qa_alt2.py \
      --model_name_or_path {checkpoint} \
      --dataset_name {dataset_name} \
      --do_train \
      --do_eval \
      --do_predict \
      --per_device_train_batch_size 40\
      --per_device_eval_batch_size 40\
      --evaluation_strategy "no" \
      --save_strategy "no" \
      --logging_strategy "epoch" \
      --learning_rate {lr} \
      --num_train_epochs 1 \
      --max_seq_length 384 \
      --doc_stride 128 \
      --output_dir {output_dir} \
      --overwrite_output_dir \
      --train_adapter \
      --adapter_config houlsby \
      --load_adapter @ukp/roberta-base_qa_squad1_houlsby

In [3]:
def run_gradual_ft_only_covid(output_dir, checkpoint, covid_val, lr, dataset_name, adapter_checkpoint):
    !python run_qa_alt2.py \
      --model_name_or_path {checkpoint} \
      --dataset_name {dataset_name} \
      --do_train \
      --do_eval \
      --do_predict \
      --per_device_train_batch_size 40\
      --per_device_eval_batch_size 40\
      --evaluation_strategy "no" \
      --save_strategy "no" \
      --logging_strategy "epoch" \
      --learning_rate {lr} \
      --max_seq_length 384 \
      --doc_stride 128 \
      --output_dir {output_dir} \
      --overwrite_output_dir \
      --train_adapter \
      --adapter_config houlsby \
      --load_adapter {adapter_checkpoint}

In [4]:
data_files = {}
data_files["train"] = covid_file

covid_qa = get_dataset(covid_file)
bio_qa = get_dataset(bio_file)

squad_qa = concatenate_datasets([squad_dataset['train'], squad_dataset['validation']])
covid_and_squad_dataset_path = "../data/full_squad_covidQA"

# squad_qa = datasets.Dataset.from_dict(squad_qa[:50])
# covid_qa = datasets.Dataset.from_dict(covid_qa[:20])
# bio_qa = datasets.Dataset.from_dict(bio_qa[:20])

Using custom data configuration default-a396986d6b3a2375
Reusing dataset squad (C:\Users\unitu\.cache\huggingface\datasets\squad\default-a396986d6b3a2375\0.0.0\cb00e306c4924563ce3d1292a1ce1b86b2753dab6285ce43c87b39c5bda3ef4e)
Using custom data configuration default-8fdbe041288a2f4d
Reusing dataset squad (C:\Users\unitu\.cache\huggingface\datasets\squad\default-8fdbe041288a2f4d\0.0.0\cb00e306c4924563ce3d1292a1ce1b86b2753dab6285ce43c87b39c5bda3ef4e)


In [5]:

covid_path = "../data/covid_data"
def train_adapter(lr):
    k_fold = 5
    for i in range(k_fold):
        covid_fold = covid_qa.shard(k_fold, i)

        covid_test = covid_fold.shard(2, 0)
        covid_val = covid_fold.shard(2, 1)
        covid_train = concatenate_datasets([covid_qa.shard(k_fold, j) for j in range(k_fold) if j != i])

        checkpoint = 'roberta-base'
        cur_dir = '../models/COVID_adapter_bioASQ_pretrained_squad1_houlsby'+ str(lr) + '/split_' + str(i)
        #full_dataset = datasets.concatenate_datasets([bio_qa, covid_train])
        make_and_save_full_dataset(covid_train, covid_val, covid_test, covid_path)
        run_gradual_ft(cur_dir, checkpoint, covid_val, lr, covid_path)

# checkpoint for squad ../models/squad_adapter_model_output \
# load_adapter ../models/bio_adapter_model_output\bio_data

In [6]:
#train_adapter(1e-5)
# train_adapter(2e-5)
#train_adapter(3e-5)
# train_adapter(5e-5)


In [7]:
def train_bio_adapt(path):
    !python run_qa_alt2.py \
      --model_name_or_path roberta-base \
      --dataset_name {path} \
      --do_train \
      --learning_rate 2e-3 \
      --per_device_train_batch_size 40\
      --per_device_eval_batch_size 40\
      --evaluation_strategy "no" \
      --save_strategy "no" \
      --logging_strategy "epoch" \
      --max_seq_length 384 \
      --doc_stride 128 \
      --output_dir ../adapter_models/bio_adapter_model_output \
      --overwrite_output_dir \
      --train_adapter \
      --adapter_config houlsby \
      --load_adapter ../adapter_models/squad_adapter_pretrain/squad

In [8]:
squad_path = '../adapter_models/squad_adapter_pretrain'
bio_path = '../adapter_models/bio_adapter_model_output'
squad_adapter_path = '../adapter_models/squad_adapter_pretrain/squad'
bio_adapter_path = '../adapter_models/bio_adapter_model_output/bio_data'
def train_adapter_base(lr, checkpoint, adapter_path, type):
    k_fold = 5
    for i in range(k_fold):
        covid_fold = covid_qa.shard(k_fold, i)

        covid_test = covid_fold.shard(2, 0)
        covid_val = covid_fold.shard(2, 1)
        covid_train = concatenate_datasets([covid_qa.shard(k_fold, j) for j in range(k_fold) if j != i])

        #make_and_save_full_dataset(covid_train, squad_qa, covid_val, covid_test, covid_and_squad_dataset_path)

        #checkpoint = 'roberta-base'
        cur_dir = '../adapter_models/adapter_COVID_'+ type +'_trained_'+ str(lr) + '/split_' + str(i)

        #log_file = open(cur_dir + "/log_file.txt","w+")
        #sys.stdout = log_file

        # squad_qa.shuffle()
        # bio_qa.shuffle()
        make_and_save_full_dataset(covid_train, covid_val, covid_test, covid_path)
        run_gradual_ft_only_covid(cur_dir, checkpoint, covid_val, lr, covid_path, adapter_path)

In [11]:
# bio_dir = '../data/bio_data'
# make_and_save_full_dataset(bio_qa,bio_qa,bio_qa, bio_dir)
# train_bio_adapt(bio_dir)

train_adapter_base(3e-5, 'roberta-base', bio_adapter_path, 'bioASQ_roberta')

08/04/2021 11:18:47 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
08/04/2021 11:18:47 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=True,
do_train=True,
eval_accumulation_steps=None,
eval_steps=500,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
greater_is_better=None,
group_by_length=False,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=3e-05,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1,
log_level=-1,
log_level_replica=-1,
log_on_each_node=True,
logging_dir=../adapte

[INFO|tokenization_auto.py:427] 2021-08-04 11:18:48,286 >> Could not locate the tokenizer configuration file, will try to use the model config instead.
[INFO|configuration_utils.py:531] 2021-08-04 11:18:48,610 >> loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at C:\Users\unitu/.cache\huggingface\transformers\733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
[INFO|configuration_utils.py:569] 2021-08-04 11:18:48,611 >> Model config RobertaConfig {
  "adapters": {
    "adapters": {},
    "config_map": {}
  },
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings":

08/04/2021 11:48:03 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
08/04/2021 11:48:03 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=True,
do_train=True,
eval_accumulation_steps=None,
eval_steps=500,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
greater_is_better=None,
group_by_length=False,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=3e-05,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1,
log_level=-1,
log_level_replica=-1,
log_on_each_node=True,
logging_dir=../adapte

[INFO|tokenization_auto.py:427] 2021-08-04 11:48:04,647 >> Could not locate the tokenizer configuration file, will try to use the model config instead.
[INFO|configuration_utils.py:531] 2021-08-04 11:48:04,974 >> loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at C:\Users\unitu/.cache\huggingface\transformers\733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
[INFO|configuration_utils.py:569] 2021-08-04 11:48:04,974 >> Model config RobertaConfig {
  "adapters": {
    "adapters": {},
    "config_map": {}
  },
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings":

08/04/2021 12:15:49 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
08/04/2021 12:15:49 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=True,
do_train=True,
eval_accumulation_steps=None,
eval_steps=500,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
greater_is_better=None,
group_by_length=False,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=3e-05,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1,
log_level=-1,
log_level_replica=-1,
log_on_each_node=True,
logging_dir=../adapte

...

[INFO|tokenization_auto.py:427] 2021-08-04 12:42:09,181 >> Could not locate the tokenizer configuration file, will try to use the model config instead.
[INFO|configuration_utils.py:531] 2021-08-04 12:42:09,500 >> loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at C:\Users\unitu/.cache\huggingface\transformers\733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
[INFO|configuration_utils.py:569] 2021-08-04 12:42:09,501 >> Model config RobertaConfig {
  "adapters": {
    "adapters": {},
    "config_map": {}
  },
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings":

08/04/2021 13:08:25 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
08/04/2021 13:08:25 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=True,
do_train=True,
eval_accumulation_steps=None,
eval_steps=500,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
greater_is_better=None,
group_by_length=False,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=3e-05,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1,
log_level=-1,
log_level_replica=-1,
log_on_each_node=True,
logging_dir=../adapte

[INFO|tokenization_auto.py:427] 2021-08-04 13:08:26,166 >> Could not locate the tokenizer configuration file, will try to use the model config instead.
[INFO|configuration_utils.py:531] 2021-08-04 13:08:26,485 >> loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at C:\Users\unitu/.cache\huggingface\transformers\733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
[INFO|configuration_utils.py:569] 2021-08-04 13:08:26,485 >> Model config RobertaConfig {
  "adapters": {
    "adapters": {},
    "config_map": {}
  },
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings":

    })
    validation: Dataset({
        features: ['answers', 'context', 'id', 'question', 'title'],
        num_rows: 201
    })
    test: Dataset({
        features: ['answers', 'context', 'id', 'question', 'title'],
        num_rows: 202
    })
})
++++++++++++++++++++++++++++++++++++++++++++++++++
                  covid_data
++++++++++++++++++++++++++++++++++++++++++++++++++
----------------------------------------------
task_name not in model.config.adapters:  True
adapter_config:  houlsby
going to show the model name:  roberta-base
----------------------------------------------
it took in the  ../adapter_models/bio_adapter_model_output/bio_data  as a valid one
08/04/2021 13:08:29 - WARNING - datasets.arrow_dataset -   Loading cached processed dataset at ../data/covid_data/train\cache-4148579a5402e593.arrow
08/04/2021 13:08:38 - WARNING - datasets.arrow_dataset -   Loading cached processed dataset at ../data/covid_data/test\cache-1917289faff2a5e1.arrow
{'loss': 0.2495, 'learning_


[INFO|trainer.py:1205] 2021-08-04 13:08:41,006 >>   Total optimization steps = 3732

  6%|5         | 215/3732 [01:24<22:48,  2.57it/s]

In [ ]:
train_adapter_base(3e-5, bio_path, bio_adapter_path, 'bioASQ')

In [12]:
train_adapter_base(3e-5, squad_path, squad_adapter_path, 'SQuAD')


 33%|###3      | 1243/3732 [08:04<16:08,  2.57it/s]
                                                   

 67%|######6   | 2487/3732 [16:08<08:04,  2.57it/s]
                                                   

100%|#########9| 3731/3732 [24:12<00:00,  2.55it/s]
                                                   

100%|##########| 3732/3732 [24:12<00:00,  2.55it/s][INFO|trainer.py:1403] 2021-08-04 13:32:53,513 >> 

Training completed. Do not forget to share your model on huggingface.co/models =)



                                                   

100%|##########| 3732/3732 [24:12<00:00,  2.57it/s]
[INFO|trainer.py:1989] 2021-08-04 13:32:53,515 >> Saving model checkpoint to ../adapter_models/adapter_COVID_bioASQ_roberta_trained_3e-05/split_4
[INFO|loading.py:59] 2021-08-04 13:32:53,515 >> Configuration saved in ../adapter_models/adapter_COVID_bioASQ_roberta_trained_3e-05/split_4\covid_data\adapter_config.json
[INFO|loading.py:72] 2021-08-04 13:32:53,533 >> Module weights saved in ..

08/04/2021 13:34:44 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
08/04/2021 13:34:44 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=True,
do_train=True,
eval_accumulation_steps=None,
eval_steps=500,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
greater_is_better=None,
group_by_length=False,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=3e-05,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1,
log_level=-1,
log_level_replica=-1,
log_on_each_node=True,
logging_dir=../adapte


 67%|######6   | 2492/3738 [17:36<08:01,  2.59it/s]
                                                   

100%|##########| 3738/3738 [26:21<00:00,  2.56it/s]
                                                   

100%|##########| 3738/3738 [26:21<00:00,  2.56it/s][INFO|trainer.py:1403] 2021-08-04 14:01:44,333 >> 

Training completed. Do not forget to share your model on huggingface.co/models =)



                                                   

100%|##########| 3738/3738 [26:21<00:00,  2.36it/s]
[INFO|trainer.py:1989] 2021-08-04 14:01:44,335 >> Saving model checkpoint to ../adapter_models/adapter_COVID_SQuAD_trained_3e-05/split_0
[INFO|loading.py:59] 2021-08-04 14:01:44,336 >> Configuration saved in ../adapter_models/adapter_COVID_SQuAD_trained_3e-05/split_0\bio_data\adapter_config.json
[INFO|loading.py:72] 2021-08-04 14:01:44,355 >> Module weights saved in ../adapter_models/adapter_COVID_SQuAD_trained_3e-05/split_0\bio_data\pytorch_adapter.bin
[INFO|loading.py:59] 2021-08-04 14:01:

08/04/2021 14:03:43 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
08/04/2021 14:03:43 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=True,
do_train=True,
eval_accumulation_steps=None,
eval_steps=500,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
greater_is_better=None,
group_by_length=False,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=3e-05,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1,
log_level=-1,
log_level_replica=-1,
log_on_each_node=True,
logging_dir=../adapte

[INFO|tokenization_utils_base.py:1651] 2021-08-04 14:03:43,245 >> Didn't find file ../adapter_models/squad_adapter_pretrain\added_tokens.json. We won't load it.
[INFO|tokenization_utils_base.py:1715] 2021-08-04 14:03:43,245 >> loading file ../adapter_models/squad_adapter_pretrain\vocab.json
[INFO|tokenization_utils_base.py:1715] 2021-08-04 14:03:43,245 >> loading file ../adapter_models/squad_adapter_pretrain\merges.txt
[INFO|tokenization_utils_base.py:1715] 2021-08-04 14:03:43,245 >> loading file ../adapter_models/squad_adapter_pretrain\tokenizer.json
[INFO|tokenization_utils_base.py:1715] 2021-08-04 14:03:43,245 >> loading file None
[INFO|tokenization_utils_base.py:1715] 2021-08-04 14:03:43,246 >> loading file ../adapter_models/squad_adapter_pretrain\special_tokens_map.json
[INFO|tokenization_utils_base.py:1715] 2021-08-04 14:03:43,246 >> loading file ../adapter_models/squad_adapter_pretrain\tokenizer_config.json
[INFO|configuration_utils.py:529] 2021-08-04 14:03:43,310 >> loading con

...


 67%|######6   | 2484/3726 [17:24<07:57,  2.60it/s]
                                                   

100%|##########| 3726/3726 [26:03<00:00,  2.60it/s]
                                                   

100%|##########| 3726/3726 [26:03<00:00,  2.60it/s][INFO|trainer.py:1403] 2021-08-04 15:32:22,384 >> 

Training completed. Do not forget to share your model on huggingface.co/models =)



                                                   

100%|##########| 3726/3726 [26:03<00:00,  2.38it/s]
[INFO|trainer.py:1989] 2021-08-04 15:32:22,387 >> Saving model checkpoint to ../adapter_models/adapter_COVID_SQuAD_trained_3e-05/split_3
[INFO|loading.py:59] 2021-08-04 15:32:22,388 >> Configuration saved in ../adapter_models/adapter_COVID_SQuAD_trained_3e-05/split_3\bio_data\adapter_config.json
[INFO|loading.py:72] 2021-08-04 15:32:22,405 >> Module weights saved in ../adapter_models/adapter_COVID_SQuAD_trained_3e-05/split_3\bio_data\pytorch_adapter.bin
[INFO|loading.py:59] 2021-08-04 15:32:

08/04/2021 15:34:20 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
08/04/2021 15:34:20 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=True,
do_train=True,
eval_accumulation_steps=None,
eval_steps=500,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,


[INFO|tokenization_utils_base.py:1651] 2021-08-04 15:34:20,529 >> Didn't find file ../adapter_models/squad_adapter_pretrain\added_tokens.json. We won't load it.
[INFO|tokenization_utils_base.py:1715] 2021-08-04 15:34:20,530 >> loading file ../adapter_models/squad_adapter_pretrain\vocab.json
[INFO|tokenization_utils_base.py:1715] 2021-08-04 15:34:20,530 >> loading file ../adapter_models/squad_adapter_pretrain\merges.txt
[INFO|tokenization_utils_base.py:1715] 2021-08-04 15:34:20,530 >> loading file ../adapter_models/squad_adapter_pretrain\tokenizer.json
[INFO|tokenization_utils_base.py:1715] 2021-08-04 15:34:20,530 >> loading file None
[INFO|tokenization_utils_base.py:1715] 2021-08-04 15:34:20,530 >> loading file ../adapter_models/squad_adapter_pretrain\special_tokens_map.json
[INFO|tokenization_utils_base.py:1715] 2021-08-04 15:34:20,530 >> loading file ../adapter_models/squad_adapter_pretrain\tokenizer_config.json
[INFO|configuration_utils.py:529] 2021-08-04 15:34:20,594 >> loading con

gradient_accumulation_steps=1,
greater_is_better=None,
group_by_length=False,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=3e-05,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1,
log_level=-1,
log_level_replica=-1,
log_on_each_node=True,
logging_dir=../adapter_models/adapter_COVID_SQuAD_trained_3e-05/split_4\runs\Aug04_15-34-20_DESKTOP-1CU6Q06,
logging_first_step=False,
logging_steps=500,
logging_strategy=IntervalStrategy.EPOCH,
lr_scheduler_type=SchedulerType.LINEAR,
max_grad_norm=1.0,
max_steps=-1,
metric_for_best_model=None,
mp_parameters=,
no_cuda=False,
num_train_epochs=3.0,
output_dir=../adapter_models/adapter_COVID_SQuAD_trained_3e-05/split_4,
overwrite_output_dir=True,
past_index=-1,
per_device_eval_batch_size=40,
per_device_train_batch_size=40,
prediction_loss_only=False,
push_to_hub=False,
push_to_hub_model_id=split_4,
push_to_hub_organization=None,
push_to_hub_token=None,
remove_unused_columns=True,
report_to=[],



        "original_ln_after": true,
        "original_ln_before": false,
        "output_adapter": true,
        "reduction_factor": 16,
        "residual_before_ln": true
      }
    }
  },
  "architectures": [
    "RobertaForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.8.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

[INFO|modeling_utils.py:1161] 2021-08-04 15:34:20,595 >> loading weights file ../adapter_models/squad_adapter_pretrain\pytorch_model.bin
[INFO|modeling_utils.py:1349] 2021-08-04 15:34:21,3